## Generate random sample (regression problem)

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import make_regression

In [3]:
X, y, coeffs = make_regression(n_samples=50, n_features=4, n_informative=4, noise=10, coef=True, random_state=42)
display(X, y)

array([[-0.75373616, -0.24538812, -0.81581028, -0.88951443],
       [-0.2257763 ,  1.46564877, -1.42474819,  0.0675282 ],
       [-0.64511975,  0.36163603,  1.53803657,  0.36139561],
       [ 0.29698467,  0.52194157,  0.34644821,  0.25049285],
       [-1.05771093, -0.01349722, -1.22084365,  0.82254491],
       [ 0.15372511, -0.88385744, -1.1429703 ,  0.05820872],
       [ 1.35624003,  0.81252582,  1.0035329 , -0.07201012],
       [ 0.47359243, -1.06230371,  1.54993441, -0.91942423],
       [-0.30921238, -0.83921752,  0.97554513,  0.33126343],
       [-0.85715756,  0.62566735,  0.48247242, -1.0708925 ],
       [ 0.17136828,  0.73846658, -0.3011037 , -0.11564828],
       [ 0.41278093,  0.96337613,  1.89679298,  0.82206016],
       [ 0.34115197, -0.07710171,  0.82718325,  0.2766908 ],
       [ 0.11092259, -0.54438272,  0.37569802, -1.15099358],
       [-0.29169375, -0.60063869,  1.85227818, -0.60170661],
       [-0.32766215, -0.70205309, -1.46351495, -0.39210815],
       [-1.76304016,  0.

array([-201.42348529,   55.01512109,   98.50233564,  108.44325693,
        -98.42574697, -132.06975033,  260.37878851,  -11.46177441,
         -9.76972843,  -45.30132996,   45.11987851,  302.30210595,
         89.36756994,  -87.67084818,   25.22917392, -226.78603055,
       -112.6428451 ,   22.04295716,  171.69681701,   -5.23980118,
        152.21031897, -163.86709178,  116.71831939, -275.09211067,
         22.77705068,  299.10909296, -199.07464079, -128.95853436,
         66.99653404,  -67.16347752,  111.26707471,  170.54175622,
       -152.07142206,   69.06492335,   26.23621154, -148.29405234,
         48.54534548,  -13.13593296,  -16.10984285,   36.34512573,
       -222.93693663, -116.35728776,  120.8347961 ,  195.98475913,
        -10.31492204, -106.18619309, -216.12065089, -228.79136805,
       -133.30173933,  -60.86663698])

In [4]:
coeffs

array([74.07686178, 87.73730719, 70.2484084 , 69.7015741 ])

In [5]:
X.shape, coeffs.shape

((50, 4), (4,))

## Create gradient descent class

In [6]:
def calculate_matrix_loss(X: np.ndarray, w: np.ndarray, y: np.ndarray) -> float:
        return np.sum((y - (X @ w)) ** 2) / 2

class GradientDescent:
    def __init__(self, dimension: int, learning_rate: float = 1e-2, iteration: int = 10000, eps: float = 1e-4) -> None:
        self.w: np.ndarray = np.random.rand(dimension)
        self.learning_rate: float = learning_rate
        self.iteration: int = iteration
        self.eps: float = eps

    def calculate_loss(self, X: np.ndarray, y: np.ndarray) -> float:
        return (y - (X @ self.w)) @ (-X)

    def calculate_gradient(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        return self.w - self.learning_rate * self.calculate_loss(X, y)

    def run(self, X: np.ndarray, y: np.ndarray) -> tuple:
        # matrix_loss = None
        next_w = self.w
        logs = pd.DataFrame(columns=['current-weights', 'next-weights', 'loss'])
        for i in np.arange(self.iteration):
            self.w = next_w
            next_w = self.calculate_gradient(X, y)
            # if calculate_matrix_loss(X, next_w, y) <= self.eps:
            #     break
            log = pd.DataFrame([[self.w, next_w, calculate_matrix_loss(X, self.w, y)]], columns=['current-weights', 'next-weights', 'loss'])
            logs = pd.concat([logs, log], sort=False, ignore_index=True)
            if np.linalg.norm(self.w - next_w, ord=2) <= self.eps:
                break
        return (self.w, logs)

## Create linear regression class

In [7]:
class LinearRegression:
    def __init__(self, iteration: int = 10000, eps: float = 1e-4) -> None:
        self.iteration: int = iteration
        self.eps: float = eps
        self.w = None
        self.logs = None
        self.loss: float = 0

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        # add a dummy feature
        X = np.column_stack([X, np.ones(X.shape[0])])
        descent = GradientDescent(dimension=X.shape[1], learning_rate=1e-2, iteration=self.iteration, eps=self.eps)
        self.w, self.logs = descent.run(X, y)
        self.loss = calculate_matrix_loss(X, self.w, y)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return X @ self.w[:-1] + self.w[-1]
        
               

## Split and train data

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)


In [17]:
regressor.logs.tail(5)

,current-weights,next-weights,loss
53,"[73.55055054677165, 91.8625814699786, 68.74280...","[73.55058005236201, 91.86277776806166, 68.7427...",2305.121938
54,"[73.55058005236201, 91.86277776806166, 68.7427...","[73.55060383310763, 91.86293622568779, 68.7427...",2305.121934
55,"[73.55060383310763, 91.86293622568779, 68.7427...","[73.55062300205842, 91.86306413688287, 68.7427...",2305.121932
56,"[73.55062300205842, 91.86306413688287, 68.7427...","[73.550638455275, 91.86316738982435, 68.742679...",2305.121930
57,"[73.550638455275, 91.86316738982435, 68.742679...","[73.55065091428067, 91.86325073777044, 68.7426...",2305.121929


## Prediction wooooah

In [22]:
print(f"Prediction: {regressor.predict(X_test)}\nOriginal: {y_test}\nLoss: {regressor.loss}")

Prediction: [ -91.37480966   41.43945148  111.52998559  -90.52563846   32.66790294
 -145.02580714 -198.68155456  279.34465311 -125.39002788   -5.82709601]
Original: [ -87.67084818   36.34512573  111.26707471 -106.18619309   22.04295716
 -133.30173933 -199.07464079  299.10909296 -152.07142206   -5.23980118]
Loss: 2305.121928781654
